# Прикладные задачи анализа данных

## Домашнее задание 4

В этой работе мы будем решать задачу рекомендаций музыки. Для этого мы реализуем метод коллаборативной фильтрации и модель со скрытыми переменными, сравним их между собой и попытаемся сделать выводы. Во всех заданиях разрешается переписывать код по своему усмотрению, предложенные шаблоны несут лишь рекомендательный характер. За дополнительные эксперименты с методами и их анализ можно получить бонусные баллы.

In [ ]:
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from typing import Callable, List

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as scs

In [ ]:
ratings = pd.read_csv('music_dataset.csv')
ratings.head()

,userId,trackId
0,0,14
1,0,95
2,0,219
3,0,220
4,0,404


In [ ]:
tracks_info = pd.read_csv('tracks_info.csv')
tracks_info.head()

,id,name,artists
0,0,What There Is,['a-ha']
1,1,I'll Play The Blues For You,['Albert King']
2,2,Breaking Up Somebody's Home,['Albert King']
3,3,Imma Be,['Black Eyed Peas']
4,4,Boom Boom Pow,['Black Eyed Peas']


Для оценки качества рекомендаций мы будем использовать метрику $MAP@k$.

$$
MAP@k = \frac{1}{N} \sum_{u = 1}^N AP_u@k
$$
$$
AP_u@k = \frac{1}{\min(k, n_u)} \sum_{i=1}^k r_u(i) p_u@i
$$
$$p_u@k = \dfrac{1}{k}\sum_{j=1}^k r_u(j)$$


*   $N$ - количество пользователей.
*   $n_u$ - число релевантных треков пользователя $u$ на тестовом промежутке.
*   $r_u(i)$ - бинарная величина: относится ли трек на позиции $i$ к релевантным.

**Задание 1 (0.5 балла).** Реализуйте метрику $MAP@k$.

In [ ]:
# https://towardsdatascience.com/mean-average-precision-at-k-map-k-clearly-explained-538d8e032d2
def mapk(relevant: List[List[int]], predicted: List[List[int]], k: int = 20):
    # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

    apks = [] # создадим список apk для всех юзеров
    
    for i in range(len(relevant)): # пройдемся по спискам для каждого юзера

      if k != 0:
        pred = predicted[i][:k] # выбираем k-рекомендаций
        
      correct_predictions = 0 # здесь будем суммировать релевантные рекомендации (бинарно – 0/1 для каждого айтема)
      running_sum = 0 # здесь будем суммировать p@k
      m = 0 

      for n, rating in enumerate(pred): # проходимся по предсказанным рекомендациям
    
        m = n + 1 # начинаем нумерацию с 1, чтобы учесть rank
    
        if rating in relevant[i]: # смотрим, есть ли предсказанная рекомендация в ground truth
          correct_predictions += 1 # если да, то она релевантна – присваиваем 1
          running_sum += correct_predictions/m # делим кол-во релевантных рекоммендаций с учетом ранжирования рекомендаций
          # то есть 2 релевантные рекомендации из 5 с rank 1 и 2 не эквивалентно ситуации 2 релевантных рекомендаций из 5 с rank 4 и 5

      apks.append(running_sum/min(len(relevant[i]), k)) # усредняем на минимум из кол-ва релевантных рекомендаций ground truth и k

    return np.mean(apks)# усредняем по всем пользователям 

In [ ]:
relevant = [
    [1, 7, 6, 2, 8],
    [1, 5, 4, 8],
    [8, 2, 5]
]

pred = [
    [8, 1, 5, 0, 7, 2, 9, 4],
    [0, 1, 8, 5, 3, 4, 7, 9],
    [9, 2, 0, 6, 8, 5, 3, 7]
]

assert round(mapk(relevant, pred, k=5), 4) == 0.4331

Разделим данные на тренировочные и тестовые так, чтобы в теcтовый датасет попали 50 последних треков каждого пользователя.

In [ ]:
def train_test_split(ratings):
    train_ratings, test_ratings = [], []
    num_test_samples = 50

    # getting train samples
    for userId, user_data in tqdm(ratings.groupby('userId')):
        train_ratings += [user_data[:-num_test_samples]]

    train_ratings = pd.concat(train_ratings).reset_index(drop=True)
    all_train_items = train_ratings['trackId'].unique()

    # getting train samples
    # we drop all tracks that are not presented it the training samples,
    # because we won't be able to learn representations for them
    for userId, user_data in tqdm(ratings.groupby('userId')):
        test_items = user_data[-num_test_samples:]
        test_items = test_items[np.isin(test_items['trackId'], all_train_items)]
        test_ratings += [test_items]

    test_ratings = pd.concat(test_ratings).reset_index(drop=True)

    return train_ratings, test_ratings

In [ ]:
train_ratings, test_ratings = train_test_split(ratings)

  0%|          | 0/241 [00:00<?, ?it/s]

  0%|          | 0/241 [00:00<?, ?it/s]

Почистим табличку с информацией о треках и закодируем id треков так, чтобы они соответствовали их порядковому номеру.

In [ ]:
redundant_rows = np.where(~np.isin(tracks_info['id'], train_ratings['trackId'].unique()))[0]
tracks_info.drop(redundant_rows, inplace=True)
tracks_info = tracks_info.reset_index(drop=True)

In [ ]:
def ids_encoder(ratings):
    users = sorted(ratings['userId'].unique())
    items = sorted(ratings['trackId'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    return uencoder, iencoder

In [ ]:
uencoder, iencoder = ids_encoder(train_ratings)
train_ratings['trackId'] = iencoder.transform(train_ratings['trackId'].tolist())
test_ratings['trackId'] = iencoder.transform(test_ratings['trackId'].tolist())
tracks_info['id'] = iencoder.transform(tracks_info['id'].tolist())

In [ ]:
train_ratings.head()

,userId,trackId
0,0,14
1,0,95
2,0,219
3,0,220
4,0,404


In [ ]:
test_ratings.head()

,userId,trackId
0,0,57582
1,0,57802
2,0,57957
3,0,58174
4,0,59168


Соберем все релевантные треки для каждого пользователя в список.

In [ ]:
test_relevant = []
test_users = []
for user_id, user_data in test_ratings.groupby('userId'):
    test_relevant += [user_data['trackId'].tolist()]
    test_users.append(user_id)

**Задание 2 (0.5 балла).** Реализуйте метод `get_test_recommendations` в классе `BaseModel`. Он принимает на вход параметр `k` и возвращает массив из `k` наиболее подходящих треков для каждого пользователя. Не забывайте удалять уже прослушанные треки из рекомендуемых.

In [ ]:
class BaseModel:
    def __init__(self, ratings: pd.DataFrame):
        self.ratings = ratings
        self.n_users = len(np.unique(self.ratings['userId']))
        self.n_items = len(np.unique(self.ratings['trackId']))

        self.R = np.zeros((self.n_users, self.n_items))
        self.R[self.ratings['userId'], self.ratings['trackId']] = 1.
        
    def recommend(self, uid: int):
        """
        param uid: int - user's id
        return: [n_items] - vector of recommended items sorted by their scores in descending order
        """
        raise NotImplementedError

    def remove_train_items(self, preds: List[List[int]], k: int):
        """
        param preds: [n_users, n_items] - recommended items for each user
        param k: int
        return: np.array [n_users, k] - recommended items without training examples
        """
        new_preds = np.zeros((len(preds), k), dtype=int)
        for user_id, user_data in self.ratings.groupby('userId'):
            user_preds = preds[user_id]
            new_preds[user_id] = user_preds[~np.in1d(user_preds, user_data['trackId'])][:k]
        return new_preds

    def get_test_recommendations(self, k: int):
        test_preds = []
        
        # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

        for user_id, user_data in self.ratings.groupby('userId'):
            test_user = self.recommend(user_id)
            test_preds.extend([test_user])
        test_user_clean = self.remove_train_items(test_preds, k)

        return test_user_clean

## Часть 1. Коллаборативная фильтрация (User2User)

Идея: чтобы выбрать треки, которые понравятся пользователю, можно набрать несколько похожих на него пользователей (соседей) и посмотреть, какие треки они слушают. После этого остается агрегировать треки этих пользователей и выбрать самые популярные. Соответственно, задача состоит из двух частей: выбора функции похожести двух пользователей и способа агрегации.

В качестве функции похожести мы будем использовать две метрики:

1. Корреляция Пирсона $$s(u, v) = \frac{\sum_{i \in I_u \cap I_v} r_{ui}r_{vi}}{\sqrt{\sum_{i \in I_u} r_{ui} ^2}\sqrt{\sum_{i \in I_v} r_{vi}^2}} $$

2. Мера Жаккара

$$ s(u, v) = \frac{|I_u \cap I_v|}{|I_u \cup I_v|} $$


Корреляция Пирсона немного видоизменена, чтобы подходить под нашу задачу.


Во всех формулах 
* $I_u$ - множество треков, прослушанных пользователем $u$.
* $r_{ui}$ - прослушал ли пользователь $u$ трек $i$ (0 или 1).

Множество соседей определим как $$N(u) = \{ v \in U \setminus \{u\} \mid s(u, v) > \alpha\},$$ где $\alpha \, - $ гиперпараметр.



Для агрегации мы будем пользоваться следующей формулой.
$$
\hat{r}_{ui} = \frac{\sum_{v \in N(u)} s(u, v) r_{vi}}{\sum_{v \in N(u)} |s(u, v)|}
$$

**Задание 3.1 (0.5 балла).** Реализуйте функцию подсчета корреляции Пирсона.

**Задание 3.2 (0.5 балла).** Реализуйте функцию подсчета меры Жаккара.

Функции принимают матрицу оценок и вектор оценок пользователя $u$ и возвращают вектор со значениями похожести пользователя $u$ на всех пользователей. Старайтесь писать оптимизированный код, за неэффективную реализацию оценка может быть снижена.

In [ ]:
def pearson(ratings: np.array, user_vector: np.array) -> np.array:
    # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

    numerator = np.dot(user_vector, ratings.T)
    denominator = np.sqrt(np.sum(user_vector ** 2)) * np.sqrt(np.sum(ratings ** 2, axis=1)).T 

    pearson_array = numerator / denominator

    return pearson_array

def jaccard(ratings: np.array, user_vector: np.array) -> np.array:
    # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

    numerator = np.dot(user_vector, ratings.T)
    denominator = np.sum(ratings, axis=1) + np.sum(user_vector) - numerator

    jaccard_array = numerator / denominator

    return jaccard_array

**Задание 4 (1 балл).** Реализуйте методы `similarity` и `recommend` класса `User2User`. `recommend` возвращает индексы треков, отсортированные в порядке убывания предсказанных оценок.


In [ ]:
class User2User(BaseModel):
    def __init__(self, ratings, similarity_func):
        super().__init__(ratings)
       
        assert similarity_func in [pearson, jaccard]

        self.similarity_func = similarity_func
        self.alpha = 0.02

    def similarity(self, user_vector: np.array):
        """
        user_vector: [n_items]
        """
        # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

        s = self.similarity_func(self.R, user_vector)
        s = np.where(s > self.alpha, s, 0)
        s = np.where(s == 1., 0, s)
        numerator = np.dot(s, self.R)
        discriminator = np.sum(s, axis = 0)

        return numerator / discriminator

    def recommend(self, uid: int):
        # your code here: (￣▽￣)/♫•*¨*•.¸¸♪
        recommendation = self.similarity(self.R[uid])

        return np.argsort(recommendation)[::-1]

**Задание 5 (1 балл).** Постройте графики зависимости значений $MAP@k$ от разных $k$ для обоих функций похожести, сравните их между собой, а также с предсказаниями случайного алгоритма и сделайте выводы.

In [ ]:
# your code here: (￣▽￣)/♫•*¨*•.¸¸♪
diff = set(test_users).symmetric_difference(set(np.arange(241)))
diff = list(diff)

model_1 = User2User(train_ratings, pearson)
forecast_pearson = model_1.get_test_recommendations(50)
predicts_pearson = np.delete(forecast_pearson, diff, axis=0)

model_2 = User2User(train_ratings, jaccard)
forecast_jaccard = model_2.get_test_recommendations(50)
predicts_jaccard = np.delete(forecast_jaccard, diff, axis=0)

forecast_random = np.random.choice(train_ratings['trackId'].unique(), (241,50))
predicts_random = np.delete(forecast_random, diff, axis=0)

x = np.arange(50) + 1
y_pearson = []
y_jaccard = []
y_random = []

for i in x:
  y_pearson.append(round(mapk(test_relevant, predicts_pearson, k=i), 4))
  y_jaccard.append(round(mapk(test_relevant, predicts_jaccard, k=i), 4))
  y_random.append(round(mapk(test_relevant, predicts_random, k=i), 4))

<ipython-input-16-3f5a6655e0f9>:22: RuntimeWarning: invalid value encountered in true_divide
  return numerator / discriminator


In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, subplot_titles=("Pearson", "Jaccard", "Random"))

fig.append_trace(go.Scatter(
    x=x,
    y=y_pearson,
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=y_jaccard,
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=y_random
), row=3, col=1)


fig.update_layout(height=1200, width=1000, title_text="Stacked Subplots")
fig.show()

Ожидаемо, что случайный алгоритм работает хуже всего (MAP@k постоянно около 0). Если сравнивать Пирсона и Жаккара, то результаты Пирсона лучше.

**Задание 6 (1.5 балла).** Как вы могли заметить, матрица оценок получается очень разреженной, но мы работаем с ней как с обычной, это не дело. Перепишите код так, чтобы все методы могли работать с разреженными матрицами и сравните скорость работы такого подхода с оригинальным.

In [ ]:
# your code here: ‿︵‿︵ヽ(°□° )ノ︵‿︵‿

class BaseModel:
    def __init__(self, ratings: pd.DataFrame):
        self.ratings = ratings
        self.n_users = len(np.unique(self.ratings['userId']))
        self.n_items = len(np.unique(self.ratings['trackId']))

        self.R = np.zeros((self.n_users, self.n_items))
        self.R[self.ratings['userId'], self.ratings['trackId']] = 1.
        
    def recommend(self, uid: int):
        """
        param uid: int - user's id
        return: [n_items] - vector of recommended items sorted by their scores in descending order
        """
        raise NotImplementedError

    def remove_train_items(self, preds: List[List[int]], k: int):
        """
        param preds: [n_users, n_items] - recommended items for each user
        param k: int
        return: np.array [n_users, k] - recommended items without training examples
        """
        new_preds = np.zeros((len(preds), k), dtype=int)
        for user_id, user_data in self.ratings.groupby('userId'):
            user_preds = preds[user_id]
            new_preds[user_id] = user_preds[~np.in1d(user_preds, user_data['trackId'])][:k]
        return new_preds

    def get_test_recommendations(self, k: int):
        test_preds = []
        
        # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

        for user_id, user_data in self.ratings.groupby('userId'):
            test_user = self.recommend(user_id)
            test_preds.extend([test_user])
        test_user_clean = self.remove_train_items(test_preds, k)

        return test_user_clean

Мы можем посмотреть глазами, насколько хорошо модель рекомендует треки. Для этого сравним уже прослушанные треки с рекомендованными и релевантными для случайного пользователя. Хорошо ли у вас получилось?

In [ ]:
model = User2User(train_ratings, pearson)
user_id = np.random.randint(0, model.n_users)

In [ ]:
listened_tracks = train_ratings[train_ratings.userId == user_id].trackId[:15]

print('Already listened tracks:')

tracks_info.loc[listened_tracks][['name', 'artists']]

Already listened tracks:


,name,artists
5907,Вот пуля просвистела…,['Чиж & Co']
13764,Going to the Run,['Furious Zoo']
17634,Черноокая,['Иван Кучин']
17866,Белая береза,['Александр Дюмин']
19417,На поле танки грохотали,['Чиж & Co']
24578,СГА - Частушки - два!,['Сектор Газовой Атаки']
25151,Это не девочка,['Те100стерон']
25155,Папа вам не мама,['Потап и Настя']
25277,Яблони,['Ляпис Трубецкой']
27086,Половинка,['Танцы Минус']


In [ ]:
preds = model.get_test_recommendations(15)

print('Predicted tracks:')

tracks_info.loc[preds[user_id]][['name', 'artists']]

Predicted tracks:


,name,artists
2814,Numb,['Linkin Park']
52701,Прятки,['HammAli & Navai']
49023,Грустный дэнс,"['Artik & Asti', 'Артём Качер']"
57221,"Девочка, танцуй",['Artik & Asti']
35656,I Got Love,"['Miyagi & Эндшпиль', 'Рем Дигга']"
53457,По кайфу,['Олег Кензов']
49577,Кукла колдуна,['Король и Шут']
805,Zombie,['The Cranberries']
59741,Поболело и прошло,['HENSY']
48182,Сансара,"['Баста', 'Скриптонит', 'Диана Арбенина', 'Сер..."


In [ ]:
test_tracks = test_ratings[test_ratings.userId == user_id].trackId[:15]

print('Test-time tracks:')

tracks_info.loc[test_tracks][['name', 'artists']]

Test-time tracks:


,name,artists
64613,Косички,['Mary Gu']
64928,Милый кудряшка,['Simbachka']
65142,Я никогда не стану феминисткой,['Nodahsa']
65169,Портрет,['Спектакль Джо']
65339,Белая ночь,"['Lvnx', 'Алина Селях']"
65483,Федерико Феллини,['Galibri & Mavik']
65493,Бесприданница,['DEAD BLONDE']
65515,Seven Nation Army,"['Gaullin', 'Julian Perretta']"
65565,Нокаут,"['Руки Вверх!', 'Клава Кока']"
65577,Sub Pielea Mea,"[""Carla's Dreams"", 'Robert Cristian']"


### Часть 2. Модель со скрытыми переменными

В этой части мы пощупаем метод рекомендаций со скрытыми переменными.
Идея: будем предсказывать оценки по формуле
$$
\hat{r}_{ui} = \langle p_u, q_u \rangle,
$$
$p_u \in \mathbb{R}^d$ и $q_i \in \mathbb{R}^d$ - латентные векторы пользователя $u$ и объекта $i$ соответственно. 

Оптимизировать мы будем MSE между истинной оценкой пользователя и предсказанной с регуляризацией
$$
L = \sum_{(u, i) \in R} (\hat{r}_{ui} - r_{ui})^2 + \lambda \left(\sum_{u \in U} \|p_u\|^2 + \sum_{i \in I} \|q_i\|^2\right)
$$

**Задание 7 (1.5 балла).** На лекции рассматривались два подхода к оптимизации параметров. Можно это делать обычным стохастческим градинтным спуском, а можно по очереди обновлять матрицы $P, Q$, и тогда получится метод Alternating Least Squares (ALS). Выведите формулы обновления параметров для обоих методов.

**LFM:**

Ответ:
На каждом шаге случайно выбирается пара $(u,i) \in \mathbb{R}$:

\\
$$
p_{uk}:= p_{uk} + ηq_{ik}(r_{ui} - \bar{r}_{u} - \bar{r}_{i} - <{p}_{u},{q}_{i}>),
$$
$$
q_{ik}:= q_{ik} + ηp_{uk}(r_{ui} - \bar{r}_{u} - \bar{r}_{i} - <{p}_{u},{q}_{i}>)$$

\\
**ALS:**

Ответ:
\
$$
p_{u} = \left(\sum_{i:∃ r_{ui}}q_{i}q_{i}^T\right)^{-1}\sum_{i:∃ r_{ui}}r_{ui}q_{i};
$$
\
$$
q_{i} = \left(\sum_{u:∃ r_{ui}}p_{u}p_{u}^T\right)^{-1}\sum_{u:∃ r_{ui}}r_{ui}p_{u};
$$
\
Источник: https://github.com/esokolov/ml-course-hse/blob/master/2020-spring/lecture-notes/lecture23-recommender.pdf

**Задание 8 (2 балла).** Реализуйте методы оптимизации параметров для обоих алгоритмов.

In [ ]:
class HiddenVars(BaseModel):
    def __init__(self, ratings, dim=32, mode='sgd'):
        super().__init__(ratings)
        self.dim = dim
        
        assert mode in ['sgd', 'als']
        self.mode = mode

        self.P = np.random.normal(size=(self.n_users, dim))
        self.Q = np.random.normal(size=(self.n_items, dim))

        self.lr = 0.003
        self.lamb = 0.01

        self.users, self.items = self.R.nonzero()


        self.mse = []

    def fit(self, num_iters=50):
        for epoch in tqdm(range(num_iters)):

            if self.mode == 'sgd':
                # your code here: (￣▽￣)/♫•*¨*•.¸¸♪

                for u, i in zip(self.users, self.items):

                  #dist_p = self.R[u, i] - np.dot(self.P[u, :], self.Q[i, :].T)

                  #dist_q = self.R[u, i] - np.dot(self.P[u, :], self.Q[i, :].T)
                  
                  self.P[u,:] += self.lr * (self.Q[i, :]*(self.R[u, i] - np.dot(self.P[u, :], self.Q[i, :].T)) - self.lamb * self.P[u,:])
                  self.Q[i,:] += self.lr * (self.P[u, :]*(self.R[u, i] - np.dot(self.P[u, :], self.Q[i, :].T)) - self.lamb * self.Q[i,:])
                 

                  #self.P[u, :] += self.lr * np.dot(self.Q[i, :],(self.R[u, i] - \
                  #np.mean(self.R[u, :]) - np.mean(self.R[:,i]) - \
                  #np.dot(self.P[u, :], self.Q[i, :].T)))

                  #self.Q[i, :] += self.lr * np.dot(self.P[u, :],(self.R[u, i] - \
                  #np.mean(self.R[u, :]) - np.mean(self.R[:,i]) - \
                 # np.dot(self.P[u, :], self.Q[i, :].T)))

                self.mse.append(np.sum(np.sum((np.dot(self.P, self.Q.T)- self.R)**2, axis=1), axis=0) + \
                self.lamb * (np.sum(np.linalg.norm(self.P,axis=1),axis=0) + np.sum(np.linalg.norm(self.Q,axis=1),axis=0)))


            elif self.mode == 'als':
                # your code here: (￣▽￣)/♫•*¨*•.¸¸♪
                
                for u in range(self.n_users):
                  
                  index = np.where(self.R[u]==1)[0]

                  self.P[u,:] = np.dot(np.dot(self.R[u,index],self.Q[index]), np.linalg.inv(np.dot(self.Q[index].T, self.Q[index]) + np.diag([self.lamb] * self.dim)))

                for i in range(self.n_items):
                  
                  index_u = np.where(self.R[:,i]==1)[0]
                  
                  self.Q[i,:] = np.dot(np.dot(self.R[index_u,i].T,self.P[index_u,:]), np.linalg.inv(np.dot(self.P[index_u].T, self.P[index_u]) + np.diag([self.lamb] * self.dim)))
    

                #self.P = np.dot(np.dot(self.R,self.Q), np.linalg.inv(np.dot(self.Q.T, self.Q) + np.diag([self.lamb] * self.dim)))
                #self.Q = np.dot(np.dot(self.R.T,self.P), np.linalg.inv(np.dot(self.P[self.mask_P].T, self.P[self.mask_P]) + np.diag([self.lamb] * self.dim)))

      
                self.mse.append(np.sum(np.sum((np.dot(self.P, self.Q.T)- self.R)**2, axis=1), axis=0) + \
                self.lamb * (np.sum(np.linalg.norm(self.P,axis=1),axis=0) + np.sum(np.linalg.norm(self.Q,axis=1),axis=0)))

    def recommend(self, uid):
        pred_rating = self.P[uid] @ self.Q.T

        return np.argsort(pred_rating)[::-1]

**Задание 9 (1 балл).** Для обоих алгоритмов подберите оптимальные значения размерности скрытого пространства $d$ и размера предсказания $k$. Как изменяется качество предсказаний с числом итераций обучения? Постройте соответствующие графики, сравните со случайным подхом и User2User, сделайте выводы. Какой алгоритм вам кажется более подходящим для данной задачи и почему?

P.S. Хотя бы один из методов обучения должен приводить к лучшим результатам в сравнении с User2User подходом.

P.P.S. Методу LFM свойственно переобучаться, поэтому при подборе параметров полезно смотреть на значения ошибки и оптимизируемой метрики на тренировочном датасете. Вы также можете менять начальную инициализацию и прочие параметры, за исключением архитектуры, на ваш вкус.

In [ ]:
# your code here: (￣▽￣)/♫•*¨*•.¸¸♪
model = HiddenVars(train_ratings)
model.fit()
forecast = model.get_test_recommendations(15)
predicts = np.delete(forecast, diff, axis=0)

x = np.arange(15) + 1
y_SGD = []

for i in x:
  y_SGD.append(round(mapk(test_relevant, predicts, k=i), 4))

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# your code here: (￣▽￣)/♫•*¨*•.¸¸♪
model_ALS = HiddenVars(train_ratings)
model_ALS.fit()
forecast_ALS = model_ALS.get_test_recommendations(15)
predicts_ALS = np.delete(forecast_ALS, diff, axis=0)

x = np.arange(15) + 1
y_ALS = []

for i in x:
  y_ALS.append(round(mapk(test_relevant, predicts_ALS, k=i), 4))

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
fig = make_subplots(rows=2, cols=1, subplot_titles=("SGD Train Loss", "ALS Train Loss"))


fig.append_trace(go.Scatter(
    y=model.mse,
), row=1, col=1)

fig.append_trace(go.Scatter(
    y=model_ALS.mse,
), row=2, col=1)

fig.update_layout(height=1200, width=1000, title_text="Train Loss")
fig.show()

In [ ]:
fig = make_subplots(rows=4, cols=1, subplot_titles=("SGD - dim=32 - 50 эпох - lr=0.003 ", "ALS - dim=128 - 10 эпох - lr=0.01", "Pearson", "Jaccard"))


fig.append_trace(go.Scatter(
    x=x,
    y=y_SGD,
), row=1, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=y_ALS,
), row=2, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=y_pearson
), row=3, col=1)

fig.append_trace(go.Scatter(
    x=x,
    y=y_jaccard
), row=4, col=1)


fig.update_layout(height=1200, width=1000, title_text="MAP@k")
fig.show()

Как изменяется качество предсказаний с числом итераций обучения? С ростом числа итераций обучения качество предсказаний улучшается. 

\

На графиках MAP@k можно заметить, что наилучшее значение метрики у ALS алгоритма. О сравнении ALS и Пирсона с Жаккаром говорить не стоит (в последних двух ничего не обучаем), тогда как, сравнивая ALS и SGD, можно заметить, что ALS справляется лучше в случае разреженных данных (наш кейс, не у всех юзер-айтем есть взаимодействие в контексте прослушанной музыки) – это видно по лоссу.

Источник: https://cs229.stanford.edu/proj2014/Christopher%20Aberger,%20Recommender.pdf

Если у вас получилось достаточно хорошее качество, то при оптимизации параметров марицы $Q$ похожим трекам стали соответствовать похожие векторы. Поэтому мы можем для любого трека найти наиболее близкие к нему в латентном пространстве и проверить степерь обученности модели вручную.

In [ ]:
#SGD 
example_trackId = tracks_info[tracks_info.name == 'Выхода нет'].iloc[0].id

preds = model.Q @ model.Q[example_trackId]
preds = preds / np.sqrt((model.Q**2).sum(axis=1) + 1e-8)

track_idxs = preds.argsort()[::-1][:15]

In [ ]:
similar_tracks = tracks_info.loc[track_idxs][['name', 'artists']]
similar_tracks['similarity'] = preds[track_idxs] / np.linalg.norm(model.Q[example_trackId])
similar_tracks

,name,artists,similarity
5512,Выхода нет,['Сплин'],1.000000
25850,I'm Not the Only One,['Morgan James'],0.677476
55752,Живу как хочу,['Слава'],0.621239
24264,Mack the Knife,"['Louis Armstrong', 'His All-Stars']",0.620978
211,It Feels So Good,['Sonique'],0.616007
34004,Whiskey Drinkin' Woman,['Nazareth'],0.615548
64681,Poolside,"['Tommy Ashby', 'Lydia Clowes', 'Sam Okell']",0.613986
54951,I Wanna Be Your Dog,['Sex Pistols'],0.601673
65863,Am Ghames,['Zviad Bekauri'],0.599984
22729,Slave,['François Feldman'],0.588443


In [ ]:
#ALS
example_trackId = tracks_info[tracks_info.name == 'Выхода нет'].iloc[0].id

preds = model_ALS.Q @ model_ALS.Q[example_trackId]
preds = preds / np.sqrt((model_ALS.Q**2).sum(axis=1) + 1e-8)

track_idxs = preds.argsort()[::-1][:15]

In [ ]:
similar_tracks = tracks_info.loc[track_idxs][['name', 'artists']]
similar_tracks['similarity'] = preds[track_idxs] / np.linalg.norm(model_ALS.Q[example_trackId])
similar_tracks

,name,artists,similarity
5512,Выхода нет,['Сплин'],1.000000
2058,Последний герой,['КИНО'],0.687144
17331,"Последнее письмо (Гудбай, Америка)",['Nautilus Pompilius'],0.597531
9689,Bicycle Race,['Queen'],0.584986
16637,Zodiac,['Zodiac'],0.584986
5153,Stormbringer,['Deep Purple'],0.584986
2179,Восьмиклассница,['КИНО'],0.577433
14871,Никто не услышит (Ой-йо),['ЧайФ'],0.572081
5570,Моё сердце,['Сплин'],0.557584
13264,Безобразная Эльза,['Крематорий'],0.549111


Кажется, что рекомендации ALS лучше в плане жанра, стиля музыки и т.д.